In [1]:
from os import environ as ENV
DATA = ENV['PWD'] + '/data/'

In [2]:
from time import time as now
import json
from itertools import combinations

from tqdm.notebook import tqdm

In [3]:
import sys
sys.path.append('..')

from circuit_postprocess import *
from should_be_stdlib import *
from neurodata import *
from circuits import *

In [4]:
from qbraid import transpile

/home/user/work/quadrigems/.venv/lib/python3.11/site-packages/qbraid/_entrypoints.py:20: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [5]:
tuning_curves_rescaled = pd.read_csv(DATA + 'data_tuning-curves_rescaled.csv', index_col=0)
tuning_curves_rescaled

,3.0,4.2,6.0,8.5,12.0,17.0,24.0,33.9,48.0
23,0.086722,0.072027,0.168098,0.108906,0.152365,0.068859,0.123657,0.089995,0.129371
32,0.078608,0.080158,0.083181,0.095927,0.076528,0.215765,0.048417,0.207027,0.114388
35,0.104947,0.109836,0.126065,0.128293,0.150514,0.091618,0.096997,0.076197,0.115534
36,0.093583,0.082544,0.090014,0.114859,0.103090,0.105004,0.112459,0.106169,0.192279
37,0.076360,0.070309,0.142336,0.141239,0.119215,0.103641,0.104605,0.129937,0.112358
...,...,...,...,...,...,...,...,...,...
424,0.067342,0.045351,0.250037,0.122910,0.115004,0.108915,0.058867,0.053632,0.177941
425,0.097614,0.075945,0.135653,0.127716,0.161124,0.093613,0.095308,0.094377,0.118650
426,0.099993,0.092348,0.164751,0.119127,0.081755,0.108576,0.106285,0.111183,0.115983
428,0.097932,0.089850,0.120197,0.121800,0.138243,0.095211,0.135635,0.111317,0.089815


In [6]:
tuning_curves_resampled = pd.read_csv(DATA + 'data_tuning-curves_resampled.csv', index_col=0)
tuning_curves_resampled

,3.0,3.6,4.3,5.2,6.3,7.6,9.1,10.9,13.2,15.8,19.0,22.9,27.6,33.2,39.9,48.0
23,0.249401,0.202876,0.215822,0.381545,0.477977,0.363946,0.328327,0.420292,0.395173,0.226299,0.234197,0.348727,0.321776,0.260525,0.283331,0.372055
32,0.210521,0.212389,0.215030,0.218653,0.225051,0.243891,0.256888,0.218355,0.255731,0.525098,0.485378,0.153590,0.255523,0.539996,0.533203,0.306344
35,0.309418,0.314312,0.325680,0.351903,0.374615,0.375823,0.387094,0.431047,0.420952,0.295464,0.272117,0.287846,0.263679,0.227092,0.247214,0.340632
36,0.270817,0.249543,0.239007,0.246740,0.267567,0.313758,0.330254,0.305926,0.298756,0.301466,0.309979,0.322530,0.323486,0.307544,0.369911,0.556431
37,0.223829,0.196587,0.213158,0.337854,0.420684,0.424024,0.403091,0.366207,0.335284,0.309221,0.299846,0.302911,0.331483,0.377468,0.378078,0.329346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424,0.175457,0.100653,0.135142,0.458584,0.638404,0.425371,0.315741,0.304242,0.296386,0.289515,0.254567,0.169513,0.127769,0.135220,0.233321,0.463616
425,0.285618,0.232911,0.227782,0.330195,0.398631,0.378123,0.388203,0.454541,0.442974,0.299799,0.275088,0.278689,0.277691,0.276072,0.294305,0.347169
426,0.294296,0.264661,0.278617,0.409499,0.478998,0.403743,0.326582,0.254932,0.253694,0.309808,0.317388,0.312215,0.318011,0.326297,0.333937,0.341359
428,0.290078,0.268255,0.269075,0.320892,0.358403,0.358273,0.368374,0.399865,0.393229,0.299162,0.309085,0.396847,0.379553,0.334062,0.298943,0.266034


# Transpile circuits

- To transpile a pennylane circuit to qasm2 (for qiskit), run it under a quantum tape, then use the qbraid transpiler
- IonQ transpiles to a dictionary which can also be saved for later

In [7]:
def circuit_to_tape(pl_circuit):
    def tape_machine(*args):
        with qml.tape.QuantumTape() as tape:
            pl_circuit(*args)
        return tape
    return tape_machine

In [8]:
tape = {
    'ang': circuit_to_tape(circuit_angle_swap),
    'ang-qft': circuit_to_tape(circuit_angle_qft_swap),
    'amp': circuit_to_tape(circuit_amp_iamp),
    'amp-qft': circuit_to_tape(circuit_amp_iamp_qft),
}

In [9]:
def small_then_big_array(data1, data2):
    if is_array_lesser(data2, data1):
        return data2, data1
    else:
        return data1, data2

In [10]:
def get_fidelity_circuits(datum, tape_machine):
    quantum_circuits = pd.DataFrame(columns = ['A', 'B', 'qasm2'])

    # A x B
    pairs, pairs_len = combinations(datum.index, 2), (len(datum) * (len(datum) - 1) // 2)
    for a, b in tqdm(pairs, total=pairs_len):
        a_i, b_i = datum.loc[a].to_numpy(), datum.loc[b].to_numpy()
        a_i, b_i = small_then_big_array(a_i, b_i)
        tape = tape_machine(a_i, b_i)
        circuit = transpile(tape, "qasm2")
        quantum_circuits.loc[len(quantum_circuits)] = [a, b, circuit]
        # quantum_fidelity.loc[len(quantum_fidelity)] = [b, a, fidelity]

    # A x A (should all be 1)
    for a in tqdm(datum.index):
        a_i = datum.loc[a].to_numpy()
        tape = tape_machine(a_i, a_i)
        circuit = transpile(tape, "qasm2")
        quantum_circuits.loc[len(quantum_circuits)] = [a, a, circuit]

    return quantum_circuits

In [11]:
get_fidelity_circuits(tuning_curves_rescaled, tape['ang']).to_excel(DATA + 'circuits_ang.xlsx')
get_fidelity_circuits(tuning_curves_rescaled, tape['ang-qft']).to_excel(DATA + 'circuits_ang-qft.xlsx')
get_fidelity_circuits(tuning_curves_resampled, tape['amp']).to_excel(DATA + 'circuits_amp.xlsx')
get_fidelity_circuits(tuning_curves_resampled, tape['amp-qft']).to_excel(DATA + 'circuits_amp-qft.xlsx')

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/2850 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]